In [1]:
import pandas as pd

# TODO :

* market basket analysis: pour determiner les corelations entre les evenements
* Variable Importance with ML Decision Tree: separer les donées en n classe pour variable interreser. n étant les velurs unique dans la classe et evaluer l'importance des valeurs

In [2]:
def add_cols(df: pd.DataFrame, cols: list, init_val):
    for col in cols:
        df[str(col)] = init_val
        
    return df



def prefix_with_col(df: pd.DataFrame) -> pd.DataFrame:
    result = add_cols(pd.DataFrame(), df.columns, '')
    result['Num_Acc'] = df['Num_Acc']
    for col in df.columns:
        if str(col) == 'Num_Acc': continue
        result[str(col)] = df[col].map(lambda x: f'{col}__{convert_to_int(x)}')
            
    return result

In [3]:
data_folder = '../data/accidents-in-france-from-2005-to-2016/'

caracteristics = pd.read_csv(data_folder + 'caracteristics.csv', encoding='latin-1', low_memory=False)

In [4]:
caracteristics = caracteristics.drop(columns=['adr', 'lat',  'long', 'gps'])

In [5]:
pd.unique(caracteristics['an'])

array([16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5])

In [6]:
users = pd.read_csv(data_folder + 'users.csv', encoding='latin-1', low_memory=False)

In [7]:
users = users.drop(columns=['num_veh'])

In [8]:
users.head(3)

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais
0,201600000001,1.0,1,1,2,0.0,11.0,0.0,0.0,0.0,1983.0
1,201600000001,1.0,1,3,1,9.0,21.0,0.0,0.0,0.0,2001.0
2,201600000002,1.0,1,3,1,5.0,11.0,0.0,0.0,0.0,1960.0


In [9]:
vehicles = pd.read_csv(data_folder + 'vehicles.csv', encoding='latin-1', low_memory=False)

In [10]:
vehicles = vehicles.drop(columns=['num_veh'])

In [11]:
vehicles.head(3)

,Num_Acc,senc,catv,occutc,obs,obsm,choc,manv
0,201600000001,0.0,7,0,0.0,0.0,1.0,1.0
1,201600000001,0.0,2,0,0.0,0.0,7.0,15.0
2,201600000002,0.0,7,0,6.0,0.0,1.0,1.0


pour analyser plus finement les données et pour augmenter les nombre les patterns on peut faire certaines pretraitemnt. notament transformer les annnée de naissance en tranche d'age. par exemple tranche d'age entre 18-25, 26-35, 36-50, 51-*.

In [12]:
dataset = caracteristics.merge(users, on='Num_Acc')
dataset = dataset.merge(vehicles, on='Num_Acc')

In [94]:
dataset.head()

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,...,actp,etatp,an_nais,senc,catv,occutc,obs,obsm,choc,manv
0,201600000001,16,2,1,1445,1,2,1,8.0,3.0,...,0.0,0.0,1983.0,0.0,7,0,0.0,0.0,1.0,1.0
1,201600000001,16,2,1,1445,1,2,1,8.0,3.0,...,0.0,0.0,1983.0,0.0,2,0,0.0,0.0,7.0,15.0
2,201600000001,16,2,1,1445,1,2,1,8.0,3.0,...,0.0,0.0,2001.0,0.0,7,0,0.0,0.0,1.0,1.0
3,201600000001,16,2,1,1445,1,2,1,8.0,3.0,...,0.0,0.0,2001.0,0.0,2,0,0.0,0.0,7.0,15.0
4,201600000002,16,3,16,1800,1,2,6,1.0,6.0,...,0.0,0.0,1960.0,0.0,7,0,6.0,0.0,1.0,1.0


In [95]:
caracteristics = None
users = None
vehicles = None

In [16]:
def user_age_range(dataset: pd.DataFrame):
    pass
    

In [17]:
def convert_to_int(val):
    try:
        return int(val)
    except ValueError:
        return -1
    
def key_gen(col_name, val):
    return f'{col_name}__{val}'

def get_unique_id(unique_ids: dict, col_name: str, val):
    k = key_gen(col_name, convert_to_int(val))
    
    if k not in unique_ids.keys():
            unique_ids[k] = len(unique_ids.keys())
            
    return unique_ids[k]
    

def transform_to_transaction(df: pd.DataFrame):
    unique_ids = {}
    
    result = add_cols(pd.DataFrame(), df.columns, -1)
    result['Num_Acc'] = df['Num_Acc']
    
    for col in df.columns:
        if str(col) == 'Num_Acc': continue
            
        result[str(col)] = df[col].map(lambda x: get_unique_id(unique_ids, str(col), x))
            
    return result, unique_ids

In [57]:
dataset_sub = dataset.iloc[:50000]

In [58]:
dataset_tr, unique_ids = transform_to_transaction(dataset_sub)

In [59]:
len(unique_ids.keys())

1646

In [60]:
dataset_tr.head(3)

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,...,actp,etatp,an_nais,senc,catv,occutc,obs,obsm,choc,manv
0,201600000001,0,1,13,44,495,500,502,511,520,...,1414,1423,1428,1529,1533,1557,1583,1601,1609,1620
1,201600000001,0,1,13,44,495,500,502,511,520,...,1414,1423,1428,1529,1534,1557,1583,1601,1610,1621
2,201600000001,0,1,13,44,495,500,502,511,520,...,1414,1423,1429,1529,1533,1557,1583,1601,1609,1620


In [22]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

te = TransactionEncoder()

In [61]:
dataset_values = dataset_tr.values

In [62]:
dataset_values = dataset_values[:, 1:] # remove Num_Acc

In [63]:
sub_sample_dataset = dataset_values[:50000]

In [64]:
sub_sample_dataset[:2]

array([[   0,    1,   13,   44,  495,  500,  502,  511,  520,  527, 1264,
        1357, 1367, 1371, 1375, 1377, 1385, 1404, 1414, 1423, 1428, 1529,
        1533, 1557, 1583, 1601, 1609, 1620],
       [   0,    1,   13,   44,  495,  500,  502,  511,  520,  527, 1264,
        1357, 1367, 1371, 1375, 1377, 1385, 1404, 1414, 1423, 1428, 1529,
        1534, 1557, 1583, 1601, 1610, 1621]])

In [65]:
te_ary = te.fit(sub_sample_dataset).transform(sub_sample_dataset)

In [105]:
df = pd.DataFrame(te_ary, columns=te.columns_)

In [106]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645
0,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [107]:
import json

def filter_binary_dataset(binary_df: pd.DataFrame, unique_ids:dict, original_df: pd.DataFrame):
    
    f = open('frequent_info.json', 'r')
    json_data = json.load(f)
    
    to_keep = []
    
    for col_name in json_data:
        col = json_data[col_name]
        
        for i in range(col['range'][0], col['range'][1]+1, 1):
            k = key_gen(col_name, i)
            if k in unique_ids:
                 to_keep.append(unique_ids[k])
    
    unknown_col = set(dataset_sub.columns) - set(json_data.keys())
    
    to_remove = []
    for col in unknown_col:
        if col == 'Num_Acc': continue
        
        unique_vals = pd.unique(original_df[col])
        for u_val in unique_vals:
            k = key_gen(col, u_val)
            if k in unique_ids:
                to_keep.append(unique_ids[k])
    
    to_keep = sorted(to_keep)
    
    return binary_df[to_keep]

In [108]:
df = filter_binary_dataset(df, unique_ids, dataset_sub)

In [109]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1573,1574,1575,1576,1577,1578,1579,1580,1581,1582
0,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [136]:
from mlxtend.frequent_patterns import association_rules, fpmax, fpgrowth

In [143]:
frequent_itemsets = fpgrowth(df, min_support=0.3, use_colnames=True)
### alternatively:
#frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
#frequent_itemsets = fpmax(df, min_support=0.6, use_colnames=True)

association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
#frequent_itemsets

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(0),(1557),1.00000,0.99324,0.99324,0.993240,1.000000,0.000000,1.000000
1,(1557),(0),0.99324,1.00000,0.99324,1.000000,1.000000,0.000000,inf
2,(0),(1414),1.00000,0.96968,0.96968,0.969680,1.000000,0.000000,1.000000
3,(1414),(0),0.96968,1.00000,0.96968,1.000000,1.000000,0.000000,inf
4,(1557),(1414),0.99324,0.96968,0.96326,0.969816,1.000140,0.000135,1.004504
...,...,...,...,...,...,...,...,...,...
4446,"(0, 501, 1367)","(512, 1557, 1414)",0.48916,0.72436,0.35366,0.722995,0.998115,-0.000668,0.995071
4447,"(501, 1414, 1367)","(512, 0, 1557)",0.48878,0.74748,0.35366,0.723557,0.967995,-0.011693,0.913460
4448,"(1557, 501, 1367)","(512, 0, 1414)",0.48766,0.72968,0.35366,0.725218,0.993886,-0.002176,0.983763
4449,"(512, 501)","(0, 1557, 1414, 1367)",0.48466,0.73532,0.35366,0.729707,0.992367,-0.002720,0.979235


In [113]:
id_inv_map = {v: k for k, v in unique_ids.items()}

In [114]:
def decompose_key(k:str):
    return k.split('__')[0], int(k.split('__')[1])

In [139]:
# save resulte

frequent_itemsets.to_csv('closed_itemset.csv', index=False)

with open('id_inv_map.json', 'w') as f:
    json.dump(id_inv_map, f)


In [140]:
f = open('frequent_info.json', 'r')
desc = json.load(f)

res = []
for i, itemset in frequent_itemsets.iterrows():
    l = [itemset['support']]
    
    it = []
    for i in itemset['itemsets']:
        col_name , val = decompose_key(id_inv_map[i])
        d = '___'
        if col_name in desc:
            d = desc[col_name]['description']
        it.append((col_name, val, d[:15]))
        
    res.append((it, l))


In [141]:
s = 0
for r in res: #[s:s + 10]:
    print(r)
    print()

([('an', 16, '___'), ('agg', 2, 'Localisation : '), ('occutc', 0, '___')], [0.31774])

([('an', 16, '___'), ('occutc', 0, '___'), ('actp', 0, ' Action of the '), ('sexe', 2, 'Sex of the user')], [0.313])

([('an', 16, '___'), ('grav', 3, ' Severity of th'), ('occutc', 0, '___'), ('actp', 0, ' Action of the ')], [0.31878])

([('an', 16, '___'), ('actp', 0, ' Action of the '), ('grav', 1, ' Severity of th'), ('int', 1, 'Type of Interse')], [0.30136])

([('an', 16, '___'), ('grav', 1, ' Severity of th'), ('occutc', 0, '___'), ('actp', 0, ' Action of the ')], [0.38002])

([('an', 16, '___'), ('trajet', 5, 'Reason for trav'), ('sexe', 1, 'Sex of the user')], [0.30052])

([('an', 16, '___'), ('trajet', 5, 'Reason for trav'), ('occutc', 0, '___'), ('lum', 1, 'Lighting : ligh')], [0.30194])

([('an', 16, '___'), ('trajet', 5, 'Reason for trav'), ('actp', 0, ' Action of the '), ('occutc', 0, '___'), ('int', 1, 'Type of Interse')], [0.31088])

([('an', 16, '___'), ('atm', 1, 'Atmospheric con'), 

In [ ]:
decrypt(closed_itemset)

In [ ]:
def decrypt(transactionsList):
    cmp = 0
    f = open("frequent_sets.txt", "w")
    
    for itemsetList in transactionsList:
        f.write("-----------------ITEM SET----------------------\n")
        for item in itemsetList:
            
            print(item)
            if cmp != 0:
                colonne = item[0]
                valeur = item[1]
                if colonne == "lum":
                    if valeur == 1:
                        f.write(colonne+" : Full day")
                    elif valeur == 2:
                        f.write(colonne+" : Twilight or dawn")
                    elif valeur == 3:
                        f.write(colonne+" : Night without public lighting")
                    elif valeur == 4:
                        f.write(colonne+" :  Night with public lighting not lit")
                    elif valeur == 5:
                        f.write(colonne+" : Night with public lighting on")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "agg":
                    if valeur == 1:
                        f.write(colonne+" : Out of agglomeration")
                    elif valeur == 2:
                        f.write(colonne+" : In built-up areas")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "int":
                    if valeur == 1:
                        f.write(colonne+" : Out of intersection")
                    elif valeur == 2:
                        f.write(colonne+" : Intersection in X")
                    elif valeur == 3:
                        f.write(colonne+" : Intersection in T")
                    elif valeur == 4:
                        f.write(colonne+" : Intersection in Y")
                    elif valeur == 5:
                        f.write(colonne+" : Intersection with more than 4 branches")
                    elif valeur == 6:
                        f.write(colonne+" : Giratory")
                    elif valeur == 7:
                        f.write(colonne+" : Place")
                    elif valeur == 8:
                        f.write(colonne+" : Level crossing")
                    elif valeur == 9:
                        f.write(colonne+" : Other intersection")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "atm":
                    if valeur == 1:
                        f.write(colonne+" : Normal")
                    elif valeur == 2:
                        f.write(colonne+" : Light rain")
                    elif valeur == 3:
                        f.write(colonne+" : Heavy rain")
                    elif valeur == 4:
                        f.write(colonne+" : Snow - hail")
                    elif valeur == 5:
                        f.write(colonne+" : Fog - smoke")
                    elif valeur == 6:
                        f.write(colonne+" : Strong wind - storm")
                    elif valeur == 7:
                        f.write(colonne+" : Dazzling weather")
                    elif valeur == 8:
                        f.write(colonne+" : Cloudy weather")
                    elif valeur == 9:
                        f.write(colonne+" : Other")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "col":
                    if valeur == 1:
                        f.write(colonne+" : Two vehicles - frontal")
                    elif valeur == 2:
                        f.write(colonne+" : Two vehicles - from the rear")
                    elif valeur == 3:
                        f.write(colonne+" : Two vehicles - by the side")
                    elif valeur == 4:
                        f.write(colonne+" : Three vehicles and more - in chain")
                    elif valeur == 5:
                        f.write(colonne+" : Three or more vehicles - multiple collisions")
                    elif valeur == 6:
                        f.write(colonne+" : Other collision")
                    elif valeur == 7:
                        f.write(colonne+" : Without collision")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "catr":
                    if valeur == 1:
                        f.write(colonne+" : Highway")
                    elif valeur == 2:
                        f.write(colonne+" : National Road")
                    elif valeur == 3:
                        f.write(colonne+" : Departmental Road")
                    elif valeur == 4:
                        f.write(colonne+" : Communal Way")
                    elif valeur == 5:
                        f.write(colonne+" : Off public network")
                    elif valeur == 6:
                        f.write(colonne+" : Parking lot open to public traffic")
                    elif valeur == 9:
                        f.write(colonne+" : other")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")

                elif colonne == "circ":
                    if valeur == 1:
                        f.write(colonne+" : One way")
                    elif valeur == 2:
                        f.write(colonne+" : Bidirectional")
                    elif valeur == 3:
                        f.write(colonne+" : Separated carriageways")
                    elif valeur == 4:
                        f.write(colonne+" : With variable assignment channels")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "vosp":
                    if valeur == 1:
                        f.write(colonne+" : Bike path")
                    elif valeur == 2:
                        f.write(colonne+" : Cycle Bank")
                    elif valeur == 3:
                        f.write(colonne+" : Reserved channel")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "Prof":
                    if valeur == 1:
                        f.write(colonne+" : Dish")
                    elif valeur == 2:
                        f.write(colonne+" : Slope")
                    elif valeur == 3:
                        f.write(colonne+" : Hilltop")
                    elif valeur == 4:
                        f.write(colonne+" : Hill bottom")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")

                elif colonne == "plan":
                    if valeur == 1:
                        f.write(colonne+" : Straight part")
                    elif valeur == 2:
                        f.write(colonne+" : Curved on the left")
                    elif valeur == 3:
                        f.write(colonne+" : Curved right")
                    elif valeur == 4:
                        f.write(colonne+" : In \"S")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")      

                elif colonne == "surf":
                    if valeur == 1:
                        f.write(colonne+" : normal")
                    elif valeur == 2:
                        f.write(colonne+" : wet")
                    elif valeur == 3:
                        f.write(colonne+" : puddles")
                    elif valeur == 4:
                        f.write(colonne+" : flooded")
                    elif valeur == 5:
                        f.write(colonne+" : snow")
                    elif valeur == 6:
                        f.write(colonne+" : mud")
                    elif valeur == 7:
                        f.write(colonne+" : icy")
                    elif valeur == 8:
                        f.write(colonne+" : fat - oil")
                    elif valeur == 9:
                        f.write(colonne+" : other")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")                
                elif colonne == "infra":
                    if valeur == 1:
                        f.write(colonne+" : Underground - tunnel")
                    elif valeur == 2:
                        f.write(colonne+" : Bridge - autopont")
                    elif valeur == 3:
                        f.write(colonne+" : Exchanger or connection brace")
                    elif valeur == 4:
                        f.write(colonne+" : Railway")
                    elif valeur == 5:
                        f.write(colonne+" : Carrefour arranged")
                    elif valeur == 6:
                        f.write(colonne+" : Pedestrian area")
                    elif valeur == 7:
                        f.write(colonne+" : Toll zone")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")   
                elif colonne == "situ":
                    if valeur == 1:
                        f.write(colonne+" : On the road")
                    elif valeur == 2:
                        f.write(colonne+" : On emergency stop band")
                    elif valeur == 3:
                        f.write(colonne+" : On the verge")
                    elif valeur == 4:
                        f.write(colonne+" : On the sidewalk")
                    elif valeur == 5:
                        f.write(colonne+" : On bike path")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")    
                elif colonne == "catu":
                    if valeur == 1:
                        f.write(colonne+" : Driver")
                    elif valeur == 2:
                        f.write(colonne+" : Passenger")
                    elif valeur == 3:
                        f.write(colonne+" : Pedestrian")
                    elif valeur == 4:
                        f.write(colonne+" : Pedestrian in rollerblade or scooter")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")                 

                elif colonne == "grav":
                    if valeur == 1:
                        f.write(colonne+" : Unscathed")
                    elif valeur == 2:
                        f.write(colonne+" : Killed")
                    elif valeur == 3:
                        f.write(colonne+" : Hospitalized wounded")
                    elif valeur == 4:
                        f.write(colonne+" : Light injury")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "sex":
                    if valeur == 1:
                        f.write(colonne+" : Male")
                    elif valeur == 2:
                        f.write(colonne+" : Female")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "trip":
                    if valeur == 1:
                        f.write(colonne+" : Home - work")
                    elif valeur == 2:
                        f.write(colonne+" : Home - school")
                    elif valeur == 3:
                        f.write(colonne+" : Shopping - Shopping")
                    elif valeur == 4:
                        f.write(colonne+" : Professional use")
                    elif valeur == 5:
                        f.write(colonne+" : Promenade - leisure")
                    elif valeur == 9:
                        f.write(colonne+" : other")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "secu":
                    val = ""
                    if int(str(valeur)[:1]) == 1:
                        val += "Belt"
                    elif int(str(valeur)[:1]) == 2:
                        val +="Helmet"
                    elif int(str(valeur)[:1]) == 3:
                        val +="Children's device"
                    elif int(str(valeur)[:1]) == 4:
                        val +="Reflective equipment"
                    elif int(str(valeur)[:1]) == 9:
                        val += "other"
                    elif int(str(valeur)[:2]) == 1:
                        val +=", Yes"
                    elif int(str(valeur)[:2]) == 2:
                        val +=", No"
                    elif int(str(valeur)[:2]) == 3:
                        val +=", Not determinable"
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                    f.write(colonne+" : "+ val)
                elif colonne == "locp":
                    print(valeur)
                    if valeur == 1:
                        f.write(colonne+" : On pavement: A + 50 m from the pedestrian crossing")
                    elif valeur == 2:
                        f.write(colonne+" : On pavement: A - 50 m from the pedestrian crossing")
                    elif valeur == 3:
                        f.write(colonne+" : On pedestrian crossing Without light signaling")
                    elif valeur == 4:
                        f.write(colonne+" : On pedestrian crossing With light signaling")
                    elif valeur == 5:
                        f.write(colonne+" : On the sidewalk")
                    elif valeur == 6:
                        f.write(colonne+" : On the verge")
                    elif valeur == 7:
                        f.write(colonne+" : On refuge or BAU")
                    elif valeur == 8:
                        f.write(colonne+" : On against aisle")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "actp":
                    if valeur == 0:
                        f.write(colonne+" : Moving, not specified or not applicable")
                    elif valeur == 1:
                        f.write(colonne+" : Moving, Meaning bumping vehicle")
                    elif valeur == 2:
                        f.write(colonne+" : Moving, Opposite direction of the vehicle")
                    elif valeur == 3:
                        f.write(colonne+" : Various, Crossing")
                    elif valeur == 4:
                        f.write(colonne+" : Various, Masked")
                    elif valeur == 5:
                        f.write(colonne+" : Various, Playing - running")
                    elif valeur == 6:
                        f.write(colonne+" : Various, With animal")
                    elif valeur == 9:
                        f.write(colonne+" : Various, Other")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "etatp":
                    if valeur == 1:
                        f.write(colonne+" : Only")
                    elif valeur == 2:
                        f.write(colonne+" : Accompanied")
                    elif valeur == 3:
                        f.write(colonne+" : In a group")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "GP":
                    if valeur == 1:
                        f.write(colonne+" : PK or PR or increasing postal address number")
                    elif valeur == 2:
                        f.write(colonne+" : PK or PR or descending postal address number")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "CATV":
                    if valeur == 1:
                        f.write(colonne+" : Bicycle")
                    elif valeur == 2:
                        f.write(colonne+" : Moped <50cm3")
                    elif valeur == 3:
                        f.write(colonne+" : Cart (Quadricycle with bodied motor) (formerly \"cart or motor tricycle\")")
                    elif valeur == 4:
                        f.write(colonne+" : Not used since 2006 (registered scooter)")
                    elif valeur == 5:
                        f.write(colonne+" : Not used since 2006 (motorcycle)")
                    elif valeur == 6:
                        f.write(colonne+" : Not used since 2006 (side-car)")
                    elif valeur == 7:
                        f.write(colonne+" : VL only")
                    elif valeur == 8:
                        f.write(colonne+" : Not used category (VL + caravan)")
                    elif valeur == 9:
                        f.write(colonne+" : Not used category (VL + trailer)")
                    elif valeur == 10:
                        f.write(colonne+" : VU only 1,5T <= GVW <= 3,5T with or without trailer (formerly VU only 1,5T <= GVW <= 3,5T)")
                    elif valeur == 11:
                        f.write(colonne+" : Most used since 2006 (VU (10) + caravan)")
                    elif valeur == 12:
                        f.write(colonne+" : Most used since 2006 (VU (10) + trailer)")
                    elif valeur == 13:
                        f.write(colonne+" : PL only 3,5T")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                else:
                    f.write(colonne+" : "+str(valeur))
                f.write("\n")
            cmp +=1
        cmp = 0

In [ ]:
decrypt(res)